In [1]:
from src import agent

# Пример использования

In [2]:
while(True):
    rq = input("\Пользователь: ")
    print("Пользователь: ", rq)
    if rq == "":
        break
    resp = agent.invoke({"messages": [("user", rq)]})
    print("Ассистент: ", resp["messages"][-1].content)

Пользователь:  Выведи сумму продаж продукта книги за каждый год 
Бот запросил информацию о таблицах
Бот запросил информацию о списке полей в таблице
Бот сделал проверил значения полей
Бот сделал запрос к базе данных
Ассистент:  Вот сумма продаж книг за каждый год:

| Год   | Сумма Продаж |
|-------|--------------|
| 2016  | 1212111      |
| 2017  | 1223458      |
| 2018  | 1581554      |
| 2019  | 1314063      |
| 2020  | 1139186      |
| 2021  | 1367635      |
| 2022  | 1490731      |
| 2023  | 1180537      |
| 2024  | 1067656      |
| 2025  | 1243947      |
Пользователь:  Для всех клиентов 2000 года рождения выведи среднюю сумму покупок для каждого типа продукта. Отсортируй по продукту
Бот запросил информацию о таблицах
Бот запросил информацию о списке полей в таблице
Бот запросил информацию о списке полей в таблице
Бот сделал проверил значения полей
Бот сделал проверил значения полей
Бот сделал запрос к базе данных
Ассистент:  Клиенты 2000 года рождения совершили следующие средние с

# Проверим корректнось полученного ответа

In [3]:
import sqlite3

In [10]:
request = '''
select product_type, avg(amount_rub)
from customers t1
join purchases t2
on t1.customer_id = t2.customer_id
where birth_year = 2000
group by 1
order by 1;
'''

conn = sqlite3.connect("db/crm.db")
cursor = conn.cursor()

cursor.execute(request)
rows = cursor.fetchall()
conn.close()

for r in rows:
    print(f"Продукт: {r[0]}, Сумма: {round(r[1], 2)} руб.")

Продукт: Книги, Сумма: 31022.0 руб.
Продукт: Мебель, Сумма: 24262.64 руб.
Продукт: Одежда, Сумма: 27510.17 руб.
Продукт: Продукты, Сумма: 24363.0 руб.
Продукт: Техника, Сумма: 29251.5 руб.
Продукт: Электроника, Сумма: 28949.25 руб.


In [7]:
request = '''
select purchase_year, sum(amount_rub) amount_rub
from purchases 
where product_type = 'Книги'
group by 1
order by 1;
'''

conn = sqlite3.connect("db/crm.db")
cursor = conn.cursor()

cursor.execute(request)
rows = cursor.fetchall()
conn.close()

for r in rows:
    print(f"Год: {r[0]}, Сумма: {r[1]} руб.")

Год: 2016, Сумма: 1212111 руб.
Год: 2017, Сумма: 1223458 руб.
Год: 2018, Сумма: 1581554 руб.
Год: 2019, Сумма: 1314063 руб.
Год: 2020, Сумма: 1139186 руб.
Год: 2021, Сумма: 1367635 руб.
Год: 2022, Сумма: 1490731 руб.
Год: 2023, Сумма: 1180537 руб.
Год: 2024, Сумма: 1067656 руб.
Год: 2025, Сумма: 1243947 руб.


Вручную полученные результаты совпадают с тем, что нам выдал бот

# Какие проблемы есть на данный момент

К сожалению, иногда бот не понимает из запроса пользователя, какую фильтрацию нужно использовать.
Если не указать явно, что книга – это один из продуктов, верный запрос может быть не составлен.

Возможные пути решения:
1) Более мощная модель
2) Использование мультиагентной системы
3) Корректировка system prompt

In [11]:
while(True):
    rq = input("\Пользователь: ")
    print("Пользователь: ", rq)
    if rq == "":
        break
    resp = agent.invoke({"messages": [("user", rq)]})
    print("Ассистент: ", resp["messages"][-1].content)

Пользователь:  Выведи сумму продаж книг за каждый год
Бот запросил информацию о таблицах
Бот запросил информацию о списке полей в таблице
Бот сделал запрос к базе данных
Бот сделал запрос к базе данных
Бот сделал запрос к базе данных
Бот сделал запрос к базе данных
Бот сделал запрос к базе данных
Ассистент:  Проблема все еще связана с некорректным способом передачи запроса. Попробую простой и прямой метод выполнения запроса:

```sql
SELECT purchase_year, SUM(amount_rub) 
FROM purchases 
WHERE product_type = 'книги' 
GROUP BY purchase_year
```

Без дополнительных символов и конструкций:

```sql
SELECT purchase_year, SUM(amount_rub) 
FROM purchases 
WHERE product_type = 'книги' 
GROUP BY purchase_year
```

Попробую напрямую:

```sql
SELECT purchase_year, SUM(amount_rub) 
FROM purchases 
WHERE product_type = 'книги' 
GROUP BY purchase_year
```

И наконец:

```sql
SELECT purchase_year, SUM(amount_rub) 
FROM purchases 
WHERE product_type = 'книги' 
GROUP BY purchase_year
```

Прямой и чисты